In [ ]:
import papermill as pm
from nbconvert import PDFExporter
import nbformat

# Define parameter sets
parameter_sets = [
    {"alpha": 0.3, "n_iterations": 150, "learning_rate": 0.02},
    {"alpha": 0.7, "n_iterations": 200, "learning_rate": 0.01},
    # Add more parameter sets as needed
]

# Create a PDF exporter
pdf_exporter = PDFExporter()
pdf_exporter.exclude_input = True  # This will hide the input cells in the PDF

# Execute the notebook for each parameter set and convert to PDF
for i, params in enumerate(parameter_sets):
    # Execute the notebook
    output_notebook = f'output_notebook_{i}.ipynb'
    pm.execute_notebook(
        'input_notebook.ipynb',
        output_notebook,
        parameters=params
    )
    
    # Read the executed notebook
    with open(output_notebook, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Convert to PDF
    pdf_data, _ = pdf_exporter.from_notebook_node(nb)
    
    # Save the PDF
    with open(f'output_notebook_{i}.pdf', 'wb') as f:
        f.write(pdf_data)